# Geocube SDK Tutorial

-------

#### Short description

This notebook introduces you to the SDK package with a complete Geocube workflow using the Python Client. You will see a typical workflow and how to parallelize an image processing algorithm.
This notebook is in two chapters. The first presents the SDK, the second is an example of a workflow.

-------

#### Requirements

- Same as [First part](Geocube-Client-SDK.ipynb#Requirements)

- The url of a [Geocube Ingester](github.com/airbusgeo/geocube-ingester.git) connected to the Geocube Server
- A Scihub account (`SCIHUB_USERNAME` and `SCIHUB_PASSWORD` environment variable) - [Installation](https://github.com/airbusgeo/geocube-ingester/blob/main/INSTALL.MD) 

- It's highly recommended to have done the other [client tutorials](https://github.com/airbusgeo/geocube-client-python/blob/main/Jupyter) and the [ingester tutorial](https://github.com/airbusgeo/geocube-ingester/blob/main/Jupyter).

-------

#### Table of content

- [Part 1 - SDK](Geocube-Client-SDK-1.ipynb#Table-of-content) (Geocube-Client-SDK-1.ipynb)
  
  
- Part 2 - Abnormal change detection
  * [Ingestion](#1---Ingestion)
  * [Consolidation](#2---Consolidation)
  * [Processing](#3---Parallel-processing-of-timeseries)
  * [Monitoring](#4---Monitoring-abnormal-changes)


#### Notebook initialisation

In [ ]:
from datetime import datetime
from shapely import geometry
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline
import geopandas as gpd
import os

from geocube import utils, entities, Consolidater, sdk

# Define the connection to the server
secure = False # in local, or true to use TLS
geocube_client_server  = os.environ['GEOCUBE_SERVER']        # e.g. 127.0.0.1:8080 for local use
geocube_client_api_key = os.environ['GEOCUBE_CLIENTAPIKEY']  # Usually empty for local use
geocube_downloader = os.environ['GEOCUBE_DOWNLOADER']

# Define the parameters to connect to the Geocube server
connection_params = sdk.ConnectionParams(geocube_client_server, secure, geocube_client_api_key)

# Optionally, use a downloader:
connection_params.downloader = sdk.ConnectionParams(geocube_downloader)

# Connect to the server
client = connection_params.new_client()

# Abnormal change detection

In July 2021, several European countries were affected by severe floods. In particular, on 16 July around Köln in Germany, many rivers overflowed and a major landslide has destroyed a quarry 20km South-West of Köln.

We will try to visualize the aftermath of these floods by looking for abnormal changes.

Since coherence detects changes, abnormality can be defined as a significant decrease in coherence compared to usual values. It can be computed with a [z-test](https://en.wikipedia.org/wiki/Z-test) using mean and standard-deviation calculated on a year of images (to prevent from seasonal effects).

<img src="images/AbnormalChangeAlgorithm.png" width=500>


To be more robust, thresholding is done with hysteresis (`skimage.filters.apply_hysteresis_threshold`). Small areas are filtered out with the `min_area` parameter.

In [ ]:
from skimage.measure import label, regionprops_table
from skimage import filters
import pandas as pd


def abnormal_changes(image, mean, std, z_score_low, z_score_high, min_area):
    z_score = ((mean-image)/std)[:,:,0]
    abnormal = filters.apply_hysteresis_threshold(z_score, z_score_low, z_score_high)
    blobs = label(abnormal)
    props = pd.DataFrame(regionprops_table(blobs, properties=['area', 'bbox']))
    
    patches = []
    changes = np.zeros_like(blobs)
    for index, blob in props[props.area>min_area].iterrows():
        width = blob['bbox-3'] - blob['bbox-1']
        height = blob['bbox-2'] - blob['bbox-0']
        patches.append((blob['bbox-1'],blob['bbox-0'], width, height))
        changes[np.where(blobs==index)] = 1
    return patches, changes

Mean and standard deviation are computed on a timeseries:

In [ ]:
def compute_statistics(timeseries):
    if len(timeseries) == 0:
        return None, None
    timeseries[np.where(timeseries==0)] = np.nan
    std  = np.nanstd(timeseries, axis=0)
    mean = np.nanmean(timeseries, axis=0)
    return std, mean

To be run by the Geocube, the process will be done in four steps:
1. **Ingestion** in the Geocube of a year-and-half SAR imagery covering the area.
2. **Consolidation** of the data to optimize the retrieval of timeseries.
3. **Batch processing** of the calculation of statistics on one-year timeseries (mean and standard deviation with `compute_statistics()`). Results will be stored in the Geocube and used as reference.
4. **Monitoring** of the area, to detect abnormal changes on the remaining half of the year (including the floods) with `abnormal_changes()`.


<img src="images/Process.png" width=800>

For the purpose of this notebook, only a small area around Köln will be processed, but it can be scaled up to the whole Germany or Europe for instance.

In [ ]:
import json
from dateutil import parser

AoiPath="inputs/koln.json"

with open(AoiPath, "r") as f:
    aoiJson = json.load(f)
    
YearRef = datetime(parser.parse(aoiJson["start_time"]).year, 1, 1)
YearCur = datetime(parser.parse(aoiJson["end_time"]).year, 1, 1)
CRS = aoiJson["graph_config"]["projection"]
Resolution = aoiJson["graph_config"]["resolution"]
RecordTags = aoiJson["record_tags"]
RecordTags["constellation"] = "SENTINEL1"
VariableName="CoherenceVV"
InstanceName="master"


## 1 - Ingestion
<img src="images/1.Ingestion.png" width=800>
The area, the time interval and all the processing parameters are defined in a JSON.

This JSON will be sent to the Ingester-Workflow.
See [Ingester](https://github.com/airbusgeo/geocube-ingester/) and [Ingestion Notebook](https://github.com/airbusgeo/geocube-ingester/blob/main/Jupyter/Geocube-Ingester-Demo.ipynb) for further details.

In [ ]:
import json

print(json.dumps(aoiJson, indent=4))

# Display AOI
plt.rcParams['figure.figsize'] = [10, 10]
aoi = utils.read_aoi(AoiPath)
world = gpd.read_file(gpd.datasets.get_path('naturalearth_lowres'))
base = world.plot(color='lightgrey', edgecolor='white')
gpd.GeoSeries(aoi, crs='epsg:4326').plot(ax=base, edgecolor='black')
plt.axis([aoi.bounds[0]-5, aoi.bounds[2]+5, aoi.bounds[1]-3, aoi.bounds[3]+3])

In [ ]:
workflow_server = os.environ.get('GEOCUBE_INGESTER_WORKFLOW')
!curl -F "area=@{AoiPath}" {workflow_server}/catalog/aoi

In [ ]:
!curl -s {workflow_server}/aoi/Koln/dot > outputs/Koln.dot

import graphviz
dot = graphviz.Source.from_file('Koln.dot', directory="outputs")
filename=dot.render(format='png')
from IPython.display import Image
with open(os.path.join(os.getcwd(), filename),'rb') as f:
    display(Image(data=f.read(), format='png', width=1024, height=1024))


Ingestion workflow
    
<img src="images/Ingestion.dot.png" width=800>

This process may last several hours or days, depending on the number, the power and the internet bandwidth of the processing machines.

## 2 - Consolidation

<img src="images/2.Consolidation.png" width=800>

The consolidation step will rewrite the datasets so that their retrieval as tiled-timeseries is more efficient. It's not mandatory, but it is very interesting in many use-cases (more details in the [Consolidation Notebook](https://github.com/airbusgeo/geocube-client-python/blob/main/Jupyter/Geocube-Client-DataConsolidation.ipynb#0---Introduction-to-Consolidation)).

### Layout
For this example, the timeseries will be processed using 256x256 tiles in the native CRS and resolution.
As the AOI is quite small, there will not have a lot of consolidation tasks (1 to 4 maximum).

The consolidation process will prepare the files to be requested as is, using a block size of 256px and a cell size of 4096px and less than 200 records. It means that the datasets will be stored as (4096,4096,200) arrays (this determines the maximum size of the file), with chunks of 256x256 pixels.

In [ ]:
from geocube import Consolidater
consolidater = Consolidater(geocube_client_server, secure, geocube_client_api_key)

layout_name = f"{CRS}_{Resolution}m"
cell_size   = 8192
layout = entities.Layout.regular(
    name=layout_name,
    crs=CRS,
    cell_size=cell_size,
    resolution=Resolution,
    block_size=256,
    max_records=200
)

try:
    consolidater.create_layout(layout)
    print("Layout created")
except utils.GeocubeError as e:
    print(e)
    
aoi = utils.read_aoi(AoiPath)

plt.rcParams['figure.figsize'] = [15, 15]
tiles = consolidater.tile_aoi(aoi, crs=CRS, resolution=Resolution, shape=(cell_size, cell_size))
gpd.GeoSeries(aoi, crs='epsg:4326').plot(ax=entities.Tile.plot(tiles), color="None", edgecolor='black')
plt.xlim([aoi.bounds[0]-3, aoi.bounds[2]+3])
plt.ylim([aoi.bounds[1]-1, aoi.bounds[3]+1])

### Records covering the area

In [ ]:
aoi     = utils.read_aoi(AoiPath)
records = consolidater.list_records(aoi=aoi, tags=RecordTags, with_aoi=True)

print('---------------------------------')
print('{} records found'.format(len(records)))
print('---------------------------------')

gpdrecords = entities.Record.list_to_geodataframe(records)
base=gpdrecords.plot(alpha=0.005)
gpd.GeoSeries(aoi, crs='epsg:4326').plot(ax=base, color="None", edgecolor='black')
plt.axis(gpdrecords.total_bounds[[0,2,1,3]]+[-0.3, 0.3, -0.1, 0.1])

### Consolidation job
Once the records and the layout have been defined, the consolidation process can be started.
It's an asynchronous process that takes minutes.

In [ ]:
import time
jobName = "ConsolidationKolnCohVV2"

# Get the variable
v = consolidater.variable(VariableName)
v.config_consolidation(entities.DataFormat("int16", 0, 1000, -32768), resampling_alg=entities.Resampling.cubic)
vi = v.instance(InstanceName)

# Start the consolidation
try:
    job = consolidater.consolidate(jobName, vi, layout_name, records, execution_level=entities.ExecutionLevel.STEP_BY_STEP_CRITICAL)
except utils.GeocubeError as e:
    if not e.is_already_exists():
        raise
    job = consolidater.job(jobName)

# Display tasks
time.sleep(10)
try:
    base = job.plot_tasks()
    gpdrecords = entities.Record.list_to_geodataframe(records)
    gpdrecords.plot(ax=base, alpha=0.005)
    gpd.GeoSeries(aoi, crs='epsg:4326').plot(ax=base, color="None", edgecolor='black')
    plt.axis(gpdrecords.total_bounds[[0,2,1,3]]+[-0.3, 0.3, -0.1, 0.1])
except:
    pass


In [ ]:
print(job.refresh())
consolidater.wait_job(job, wait_secs=1)

## 3 - Parallel processing of timeseries

Data is ready to be retrieved as deep timeseries.
<img src="images/3.Processing.png" width=800>

The timeseries will be processed on a annual basis and the statistics will be stored in a record named after the date of the year.

Two variables are created to define the `Mean` and the `StandardDeviation` as `float32`. To distinguish between annual statistics and others, an instance named `annual` is created.



In [ ]:
StdVarName = f"{VariableName}_StandardDeviation"
MeanVarName = f"{VariableName}_Mean"
RecordYRef = f"Koln_{YearRef.year}"

# Create variables
print(f"Create variable {StdVarName} and {MeanVarName}")
client.create_variable(StdVarName, "float32", [''], exist_ok=True).instantiate('annual', {})
client.create_variable(MeanVarName, "float32", [''], exist_ok=True).instantiate('annual', {})
        
# Create records
print(f"Create record {RecordYRef} for the year {YearRef.year}")
aoi = utils.read_aoi(AoiPath)
aoi_id = client.create_aoi(aoi, exist_ok=True)
r=client.create_record(aoi_id, RecordYRef, {}, YearRef, exist_ok=True)
print("done")


### Compute statistics of the year of reference
`compute_statistics` will be executed on the timeseries of the year of reference and the results will be indexed in the geocube. `compute_and_index` is in charge of this.

It takes as parameter: `connection_params` to connect to the client, a `record_name` and other standard parameters in input to be consistent with `sdk.cube_callback_t`. 

In [ ]:
os.makedirs('outputs/'+StdVarName, exist_ok=True)
os.makedirs('outputs/'+MeanVarName, exist_ok=True)

def compute_and_index(connection_params, timeseries, crs, transform, record):
    # Compute statistics
    std, mean = compute_statistics(timeseries)
    if std is None:
        return
    
    # Index images
    client = connection_params.new_client()      
    filename = "_".join([str(x) for x in transform.to_gdal()])+".tiff"
    index(client, StdVarName, "annual", record, transform, crs, std, os.path.join(os.getcwd(), 'outputs', StdVarName, filename))
    index(client, MeanVarName, "annual", record, transform, crs, mean, os.path.join(os.getcwd(), 'outputs', MeanVarName, filename))
    return "Indexed !"
    
    
def index(client, variable, instance, record, transform, crs, image, image_path):
    variable = client.variable(variable).instance(instance)
    utils.image_to_geotiff(image, transform, crs, variable.dformat.no_data, image_path)
    client.index([entities.Container(image_path, True,
                                     [entities.Dataset(record, variable)])])


As the AOI could be very large, `compute_and_index` will be executed on tiles (covering the AOI) and processed using `sdk.get_cube()` (see [Catalogue](Geocube-Client-SDK-1.ipynb/#Catalogue-functions)) in parallel.

In [ ]:
import geopandas as gpd
from geocube import utils, entities

aoi   = utils.read_aoi(AoiPath)
shape = (256, 256)
tiles = client.tile_aoi(aoi, crs=CRS, resolution=Resolution, shape=shape)

plt.rcParams['figure.figsize'] = [15, 15]
gpd.GeoSeries(aoi, crs='epsg:4326').plot(ax=entities.Tile.plot(tiles), color="None", edgecolor='black')

In [ ]:
import functools
from geocube import sdk

# Load the records and the variable
from_time = datetime(YearRef.year, 1, 1)
to_time   = datetime(YearRef.year, 12, 31)
records   = client.list_records(aoi=aoi, from_time=from_time, to_time=to_time, tags=RecordTags)
variable  = client.variable(VariableName).instance(InstanceName)
recordRef = client.list_records(name=RecordYRef)[0]

# For each tile, define the parameters of `sdk.get_cube()`
pool_params = {f"tile{i}": {
    "connection_params": connection_params,
    "cube_params": entities.CubeParams.from_tile(tile=t, records=records, instance=variable),
    "cube_callback": functools.partial(compute_and_index,
                                       connection_params=connection_params,
                                       record=recordRef)
} for i, t in enumerate(tiles)}

print(f"{len(pool_params)} tasks defined")

Parallel computing using `dask`:

In [ ]:
import dask
from dask.distributed import Client, wait
from tqdm import tqdm
import time

tasks = [dask.delayed(sdk.get_cube)(**params) for params in pool_params.values()]
cluster = None

try:
    cluster = Client(n_workers=2, threads_per_worker=1)
    futures = cluster.compute(tasks)

    with tqdm(total=len(futures)) as pbar:
        while len(futures) > 0:
            nf = []
            for f in futures:
                if f.status=="finished":
                    pbar.update(1)
                else:
                    nf.append(f)
                    if f.status == "error":
                        print("retry", f, f.exception())
                        f.retry()
            time.sleep(0.5)
            futures = nf
    print("done")
finally:
    if cluster is not None:
        cluster.close()

### Visualize statistics
Before last step, we check that the annual statistics computed are correct:

In [ ]:
plt.rcParams['figure.figsize'] = [15, 50]

record = client.list_records(RecordYRef)[0]
tile = entities.Tile.from_aoi(aoi, crs=CRS, resolution=20)
cp = entities.CubeParams.from_tile(tile, records=[record], instance=None)

cp.instance = client.variable(MeanVarName).instance("annual")
cube_it = client.get_cube_it(cp)
plt.subplot(1, 2, 1).set_axis_off()
plt.imshow(next(cube_it)[0][...,0], cmap="gray", vmin=0, vmax=1)
plt.title(MeanVarName)
cube_it.metadata().info()

cp.instance = client.variable(StdVarName).instance("annual")
images, _ = client.get_cube(cp)
plt.subplot(1, 2, 2).set_axis_off()
plt.imshow(images[0][...,0], cmap="gray", vmin=0, vmax=1)
_ =plt.title(StdVarName)


## 4 - Monitoring abnormal changes
As we want to call `abnormal_changes()` on all the 2021 images, several months after the events, the whole timeseries will be retrieved at once.
But for a monitoring service, only the newest images are processed. In that case, the consolidation step may not be necessary (at least for the "timeseries" perspective).

<img src="images/4.Monitoring.png" width=800>

The function `abnormal_changes_on_tile` downloads the 2021 timeseries and the 2020 statistics on a given tile, call `abnormal_changes()` for each time step and display the result.

In [ ]:
import copy
from matplotlib.patches import Rectangle

def imshowgray(ax, img, title, **kwargs):
    if "vmax" not in kwargs:
        kwargs["vmax"] = 1
    ax.imshow(img, cmap='gray', vmin=0, **kwargs)
    ax.set_title(title)
    ax.set_axis_off()

def abnormal_changes_on_tile(connection_params, cube_params, record_ref, z_score_low, z_score_high, min_area):
    client = connection_params.new_client()
    
    # Load statistics on cube_params.Tile
    cp = entities.CubeParams.from_tile(cube_params, records=[record_ref], instance=None)
    cp.instance = client.variable(MeanVarName).instance("annual")
    mean, _ = client.get_cube(cp, verbose=False)

    cp.instance = client.variable(StdVarName).instance("annual")
    std, _ = client.get_cube(cp, verbose=False)
    
    nimages = len(cube_params.records)+1
    fig, axs = plt.subplots(nimages, 2, constrained_layout=True)
    imshowgray(axs[0][0], mean[0][..., 0], "Mean 2020")
    imshowgray(axs[0][1], std[0][..., 0], "Standard deviation 2020")
    
    for i, (image, metadata, err) in enumerate(client.get_cube_it(cube_params)):
        if err is not None:
            continue
        imshowgray(axs[i+1][0], image[:,:,0], f"{VariableName} {metadata.min_date}")
        imshowgray(axs[i+1][1], ((mean[0]-image)/std[0])[...,0], f"Z-Score {metadata.min_date}", vmax=4)

        patches, changes = abnormal_changes(image, mean[0], std[0], z_score_low, z_score_high, min_area)
        for patch in patches:
            axs[i+1][1].add_patch(Rectangle((patch[0], patch[1]), patch[2], patch[3], edgecolor='r', facecolor='none'))


The tiling for monitoring can be different from the one for preprocessing (usually, system requirements are different).

For instance, monitoring can be done in a webmercator grid at level 12 (resolution~=40 at equador).

In [ ]:
plt.rcParams['figure.figsize'] = [15, 60]
recordYRef = client.list_records(name=RecordYRef)[0]

# Define the tiling
aoi   = utils.read_aoi(AoiPath)
layout = entities.Layout.web_mercator("", z_level=12)
tiles = client.tile_aoi(aoi, layout=layout)

from_time = datetime(YearCur.year, 6, 1)
to_time   = datetime(YearCur.year, 7, 31)
records = client.list_records(aoi=aoi, from_time=from_time, to_time=to_time, tags=RecordTags)
records = entities.Record.group_by(records, entities.Record.key_date)
variable = client.variable(VariableName).instance(InstanceName)

# Compute the tile where the landslide happened
cp = entities.CubeParams.from_tile(tiles[40], records=records, instance=variable)

abnormal_changes_on_tile(connection_params, cp, recordYRef, 1.7, 3, 100)

## Conclusion

The Z-score on the coherence image computed on 21 July (coherence between 15 and 21) shows a lot of abnormal changes. One of the patches is located where the landslide happened (south-west from the center of the image). I don't know if it's really the landslide or something else and, actually, this is not the place to discuss the efficiency of this algorithm :)

In fact, the important thing to remember is that the Abnormal Change Detection Algorithm was easily connected to the Geocube, ready to scale-up and the processing was parallelized using the `sdk` package compatible with any multiprocess framework like `dask`.

This concludes the Geocube tutorial. I hope this gives you lots of ideas for using your new geocube.

Do not hesitate to contact the Geocube team via the [github](https://www.github.com/airbusgeo/) and to have a look at the other projects.
